In [1]:
import pandas as pd
train_df = pd.read_csv('train.csv',encoding='utf8')
test_df = pd.read_csv('test.csv',encoding='utf8')
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [2]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [3]:
med = train_df.median()
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)
med

PassengerId    446.0000
Survived         0.0000
Pclass           3.0000
Age             28.0000
SibSp            0.0000
Parch            0.0000
Fare            14.4542
dtype: float64

In [4]:
most = train_df['Embarked'].value_counts().idxmax()
train_df['Embarked'] = train_df['Embarked'].fillna(most)
test_df['Embarked'] = test_df['Embarked'].fillna(most)
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [5]:
n = 'Braund, Mr. Owen Harris'
n = n.split(',')[-1].split('.')[0]
n.strip()

'Mr'

In [6]:
def nameflow(n):
    n = n.split(',')[-1].split('.')[0]
    return n.strip()
mid = train_df['Name'].apply(nameflow).value_counts()
reserved = mid[mid > 30].index
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [7]:
def nameflow2(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in reserved:
        return n
    else:
        return None
train_df["Name"] = train_df["Name"].apply(nameflow2)
test_df["Name"] = test_df["Name"].apply(nameflow2)

In [8]:
x_train_nodrop = pd.get_dummies(train_df, 
                                columns=["Name", "Sex", "Embarked"])
x_test_nodrop = pd.get_dummies(test_df, 
                                columns=["Name", "Sex", "Embarked"])

In [9]:
x_train = x_train_nodrop.drop(['PassengerId','Survived','Cabin','Ticket'],axis=1)
y_train = x_train_nodrop['Survived']
x_test = x_test_nodrop.drop(['PassengerId','Cabin','Ticket'],axis=1)
testid = x_test_nodrop['PassengerId']
testid

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    'n_estimators':range(40,50),
    'max_depth':range(8,10)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10,n_jobs=4)
cv.fit(x_train, y_train)
print(cv.best_params_)
print(cv.best_score_)

{'max_depth': 9, 'n_estimators': 44}
0.8373158551810237


In [11]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=47, max_depth=9)
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print(score)
print(np.average(score))

[0.77777778 0.80898876 0.74157303 0.86516854 0.91011236 0.84269663
 0.85393258 0.7752809  0.87640449 0.82022472]
0.8272159800249688


In [12]:
clf = RandomForestClassifier(n_estimators=47, max_depth=9)
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    'PassengerId':testid,
    'Survived':pre
})
result_df.to_csv("titanic.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [13]:
pd.DataFrame({
    'column':x_train.columns,
    'importance':clf.feature_importances_
})

,column,importance
0,Pclass,0.096362
1,Age,0.143033
2,SibSp,0.061533
3,Parch,0.034768
4,Fare,0.180490
5,Name_Master,0.019400
6,Name_Miss,0.025854
7,Name_Mr,0.158693
8,Name_Mrs,0.036943
9,Sex_female,0.094314


In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)
x_train_scale

array([[1.        , 0.27117366, 0.125     , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.4722292 , 0.125     , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.32143755, 0.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [1.        , 0.34656949, 0.125     , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.32143755, 0.        , ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.39683338, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [15]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
params = {
    'n_neighbors':range(5,100)
}
cv = GridSearchCV(clf, params, n_jobs=4)
cv.fit(x_train_scale,y_train)
print(cv.best_params_)
print(cv.best_score_)

{'n_neighbors': 14}
0.8204381394764922


In [16]:
result_df = pd.DataFrame({
    'PassengerId':testid,
    'Survived':pre
})
result_df.to_csv("titanic1.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
